# Install Offline MMDetection

In [1]:
%%time
!pip install -qqq /kaggle/input/mmdetv3-env/archive/addict-2.4.0-py3-none-any.whl
!pip install -qqq /kaggle/input/mmdetv3-env/archive/mmengine-0.7.4-py3-none-any.whl
!pip install -qqq /kaggle/input/mmdetv3-env/archive/mmcv-2.0.0-cp310-cp310-linux_x86_64.whl
!pip install -qqq /kaggle/input/mmdetv3-env/archive/terminaltables-3.1.10-py2.py3-none-any.whl
!pip install -qqq /kaggle/input/pycocotools-206/wheels/pycocotools-2.0.6-cp310-cp310-linux_x86_64.whl
#!pip install -qqq /kaggle/input/weighted-boxes-fusion/ensemble_boxes-1.0.9-py3-none-any.whl
!pip install -qqq /kaggle/input/mmdetection-3-1-evn/src/mmdet-3.1.0-py3-none-any.whl

CPU times: user 2.68 s, sys: 463 ms, total: 3.15 s
Wall time: 3min 14s


In [2]:
from itertools import groupby
from pycocotools import mask as mutils
from pycocotools.coco import COCO
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import wandb
from PIL import Image
import gc

from glob import glob
import matplotlib.pyplot as plt


import base64
import numpy as np
from pycocotools import _mask as coco_mask
from typing import Text, Dict, Tuple
import zlib

import mmdet, mmcv, mmengine
from mmengine.config import Config
from mmengine.runner import Runner
from mmdet.utils import register_all_modules
from mmdet.apis import init_detector, inference_detector
from mmengine.visualization import Visualizer


print(mmdet.__version__)
print(mmcv.__version__)
print(mmengine.__version__)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


3.1.0
2.0.0
0.7.4


In [3]:
import base64
import numpy as np
from pycocotools import _mask as coco_mask
import typing as t
import zlib

def encode_binary_mask(mask: np.ndarray) -> Text:
    """Converts a binary mask into OID challenge encoding ascii text."""

    # check input mask --
    if mask.dtype != bool:
        raise ValueError(
            "encode_binary_mask expects a binary mask, received dtype == %s" %
            mask.dtype)

    mask = np.squeeze(mask)
    if len(mask.shape) != 2:
        raise ValueError(
            "encode_binary_mask expects a 2d mask, received shape == %s" %
            mask.shape)

    # convert input mask to expected COCO API input --
    mask_to_encode = mask.reshape(mask.shape[0], mask.shape[1], 1)
    mask_to_encode = mask_to_encode.astype(np.uint8)
    mask_to_encode = np.asfortranarray(mask_to_encode)

    # RLE encode mask --
    encoded_mask = coco_mask.encode(mask_to_encode)[0]["counts"]

    # compress and base64 encoding --
    binary_str = zlib.compress(encoded_mask, zlib.Z_BEST_COMPRESSION)
    base64_str = base64.b64encode(binary_str)
    return base64_str

In [4]:
# def get_glomerulus_mask(annotations: Dict, mask_shape: Tuple = (512, 512)) -> np.ndarray:
#     """ Converts glomerulus labels into boolean mask """
#     mask = np.ones(shape=mask_shape, dtype=np.uint8)
    
#     for annotation in annotations: 
#         if annotation['type'] == 'glomerulus':            
#             coords = np.array(annotation['coordinates'])
#             cv2.fillPoly(mask, pts=coords, color=0)

#     return mask.astype(bool)

In [5]:
# def morphological_gradient(binary_mask):
#     if binary_mask.dtype != np.uint8:
#         binary_mask = binary_mask.astype(np.uint8)
#     kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
#     gradient = cv2.morphologyEx(binary_mask, cv2.MORPH_GRADIENT, kernel)

#     return gradient



In [6]:
def convert_to_binary_mask(mask):
    binary_mask = (mask > 0)
    return binary_mask

In [7]:
def convert_to_mask(binary_mask):
    mask = binary_mask.astype(np.uint8) * 255
    return mask

In [8]:
path_list = ["/kaggle/input/hubmap-hacking-the-human-vasculature/test/"+i for i in os.listdir("/kaggle/input/hubmap-hacking-the-human-vasculature/test/")]

In [9]:
# img = mmcv.imread("/kaggle/input/hubmap-hacking-the-human-vasculature/test/72e40acccadf.tif")
checkpoint_file = "/kaggle/input/hubmap-final-r152-model/best_coco_segm_mAP_epoch_19.pth"
config = '/kaggle/input/hubmap-final-r152-model/my_cmrcnn.py'
cfg = Config.fromfile(config)

model = init_detector(cfg, checkpoint=checkpoint_file, device="cuda:0")
result = inference_detector(model, imgs=path_list)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

Loads checkpoint by local backend from path: /kaggle/input/hubmap-final-r152-model/best_coco_segm_mAP_epoch_19.pth
08/01 06:26:16 - mmengine - WARNING - blood_vessel is not a meta file, simply parsed as meta information


In [10]:
# result

In [11]:
ids = []
heights = []
widths = []
prediction_string = []

for res in result:
    pr_i = res.pred_instances
    id = res.img_path.split("/")[-1][:-4]

    scores = pr_i.scores.cpu().numpy()
    bboxes = pr_i.bboxes.cpu().numpy()
    masks = pr_i.masks.cpu().numpy()
    labels = pr_i.labels.cpu().numpy()
    
    height,width = (512, 512)
    
    
    pred_strings = []

    for label, score, mask in zip(labels,scores, masks):
        
        pred_strings_txt = ""
#         kernel = np.ones(shape=(3,3), dtype=np.uint8)
#         mask = cv2.dilate(mask.astype(np.uint8), kernel, 4)

        pred_strings_txt += f"0 {score} {encode_binary_mask(mask.astype(np.bool_)).decode('utf-8')}"
        pred_strings.append(pred_strings_txt)
        
    ids.append(id)
    heights.append(height)
    widths.append(width)
    prediction_string.append(" ".join(pred_strings))

sub = pd.DataFrame({"id": ids, "height": heights, "width": widths, "prediction_string": prediction_string})
sub = sub.set_index("id")
sub.to_csv("submission.csv")

In [12]:
sub

,height,width,prediction_string
id,,,
72e40acccadf,512,512,0 0.9781558513641357 eNqLjggyNU+zN/Mx9Df0N/IDk...


In [13]:
# aug_mask = [mask[i] for mask in aug_masks]
# merged_masks = merge_aug_masks(aug_mask)
# segm_result = self.mask_head[-1].get_seg_masks( merged_masks, _bboxes[i], det_labels[i])